# CIS 5450 Homework 3: Spark SQL

### **Deadline: Tuesday, 21st March, 2023 @ 10pm EST**

#### **Worth 100 points**

Welcome to CIS 5450 Homework 3! In this homework you will gain a mastery of using Spark SQL. By the end, you'll be a star (not that you aren't already one). Over the next few days you will be using an EMR cluster to use Spark to manipulate the datasets about Amazon products and their reviews. 

## The Necessary Notes and Nags
Before we begin here are some important notes to keep in mind,


1.   **IMPORTANT!** I said it twice, it's really important. In this homework, we will be using AWS resources. You are given a quota ($100) to use for the entirety of the homework. There is a small chance you will use all this money, however it is important that at the end of every session, you **shut down your EMR cluster**.
2.   **Be sure you use Google Colab for this Homework** since we must connect to the EMR cluster and local Jupyter will have issues doing that. Using a Google Colab Notebook with an EMR cluster has two important abnormalities:
    * The first line of any cell in which you will use the spark session must be `%%spark`. Notice that all cells below have this.
    * You will, unfortunately, not be able to stop a cell while it is running. If you wish to do so, you will need to restart your cluster. See the Setup EMR Document for reference.
3.   You are **required** to use Spark SQL queries to handle the data in the assignment. Mastering SQL is more beneficial than being able to use Spark commands (functions) as it will show up in more areas of programming and data science/analytics than just Spark. Use the following [function list](https://spark.apache.org/docs/latest/api/sql/index.html#) to see all the SQL functions avaliable in Spark.
4.   Throughout the homework you will be manipulating Spark dataframes (sdfs). 
5. Based on the challenges you've faced in the previous homework, we are including information on the expected schema of your results.  Apache Spark is very fiddly but we hope this will help.
6. There are portions of this homework that are _very_ hard. We urge you start early to come to office hours and get help if you get stuck. But don't worry, I can see the future, and you all got this.

With that said, let's dive in.




## Step 0: Set up EMR

Follow the [AWS Academy Getting Started](https://docs.google.com/document/d/1JPitLGaorjTbXjGsaoIHcLTu2cj8rjm5UNr9bSpZ72k/edit?usp=sharing) instructions.

Move on to Step 0.1 after you have completed all the steps in the document.

![ACME GIANT RUBBER BAND](https://pbs.twimg.com/media/DRqbJh7UMAE2z4o?format=jpg&name=4096x4096)


### 0.1: The Superfluous Setup

Run the following two cells. These will allow your colab notebook to connect to and use your EMR.

In [ ]:
#%%capture
!apt install libkrb5-dev
!pip install sparkmagic
!pip install -i https://test.pypi.org/simple/ penn-grader==0.5.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libkrb5-dev is already the newest version (1.17-6ubuntu4.2).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://test.pypi.org/simple/, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%load_ext sparkmagic.magics

The sparkmagic.magics extension is already loaded. To reload it, use:
  %reload_ext sparkmagic.magics


### 0.2: The Sharp Spark

Now, connect your notebook to the EMR cluster you created. In the first cell, copy the link to the Master Public DNS specified in the setup document. You will need to add `http://` to the beginning of the address and the auth details to the end.

For example, if my DNS (directly from the AWS EMR console) is `ec2-3-15-237-211.us-east-2.compute.amazonaws.com` my address would be,

`http://ec2-3-15-237-211.us-east-2.compute.amazonaws.com -a cis545-livy -p password1 -t Basic_Access`

Insert this in the `# TODO # below`. For our example, the cell would read,

```
%spark add -s spark_session -l python -u http://ec2-3-15-237-211.us-east-2.compute.amazonaws.com -a cis545-livy -p password1 -t Basic_Access
```

In [ ]:
# TODO: Copy the line above, enter your Master Public DNS with the proper formatting and host, and update the password

%spark add -s spark_session -l python -u http://ec2-54-226-148-91.compute-1.amazonaws.com -a cis545-livy -p Ypl14169064 -t Basic_Access

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1679003127488_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


In [ ]:
# If you ever need to restart, you may need to...
#%spark delete -s spark_session
#OR just factory reset runtime under the runtime tab
# %spark delete -s spark_session

Enter your 8-digit Penn Key as an integer in the cell 
below. This will be used in the autograder.  **Please also update the cell below, with the same ID!**

In [ ]:
from penngrader.grader import *
STUDENT_ID = 99999

In [ ]:
%%writefile config.yaml

grader_api_url: 'https://wyv616tp17.execute-api.us-east-1.amazonaws.com/default/Grader'
grader_api_key: 'Kd32fl3g3p917iM0zwjiO23Bitj4PO9ga4LektOa'

Overwriting config.yaml


In [ ]:
grader = PennGrader('config.yaml', 'CIS5450_23Sp_HW3', STUDENT_ID, STUDENT_ID)

PennGrader initialized with Student ID: 53271889

Make sure this correct or we will not be able to store your grade


Run the above cells to setup the autograder. Make sure to have set your 8 digit Penn ID in the cell above. It will also import all the modules you need for the homework.


## Step 1: Data Wrangling, Cleaning, and Shaping [32 pts]

In this homework we will be working with two datasets - (1) LinkedIn data containing information on their users like education, experience, industry etc. (2) Stock price information of companies over a 10 year period (2000-2011) where these users have worked at.  


The data you will use is stored in an S3 bucket, a cloud storage service. Below, with our help, you will download it onto the nodes of your [EMR cluster](https://docs.aws.amazon.com/emr/latest/ManagementGuide/emr-what-is-emr.html).

### 1.1: The Stupendous Schema

When loading data, Spark will try to infer its structure on its own. This process is faulty because Spark will sometimes infer the type incorrectly. Spark's ability to determine types is not reliable, thus you will need to define a schema for both the LinkedIn and Stock Prices Datasets.

A schema is a description of the structure of data. In Spark, schemas are defined using a `StructType` object. This is a collection of data types, termed `StructField`'s, that specify the structure and variable type of each component of the dataset. For example, suppose we have the following simple JSON object,


```
{
 "student_name": "Alpha Beta",
 "GPA": 3.6,
 "courses": [
    {"department": "Computer and Information Science",
     "course_id": "CIS 5450",
     "semester": "Fall 2021"},
    {"department": "Computer and Information Science",
     "course_id": "CIS 5550",
     "semester": "Fall 2021"}
 ],
 "grad_year": 2023
 }
```

We would define its schema as follows,

```       
schema = StructType([
           StructField("student_name", StringType(), nullable=True),
           StructField("GPA", FloatType(), nullable=True),
           StructField("courses", ArrayType(
                StructType([
                  StructField("department", StringType(), nullable=True),
                  StructField("course_id", StringType(), nullable=True),
                  StructField("semester", StringType(), nullable=True)
                ])
           ), nullable=True),
           StructField("grad_year", IntegerType(), nullable=True)
         ])
```


Each `StructField` has the following structure: `(name, type, nullable)`. The `nullable` flag defines that the specified field may be empty. Your first task is to define the `schema` of `linkedin_small_real.json`. A smaller version of the JSON dataset can be found [here](https://drive.google.com/a/seas.upenn.edu/file/d/1yZ_0xz6uSJ8lAxhGzn2BVjCpDOjagcqb/view?usp=sharing). Look at how this json dataset looks, the fields and their types.



You will now be defining an explicit schema for the `linkedin_small_real.json` dataset. We have defined most of the fields so you can compare how the schema looks with the JSON dataset. Your Task will be to define the schema for the following 3 fields - `name, experience, events`.

Make sure to use `nullable=True` for all the fields as well as **store dates as a StringType()**.

Note: There is also no grading cell for this step. But your JSON file won't load if it's wrong, so you have a way of testing.



In [ ]:
%%spark

from pyspark.sql.types import *

# TODO: Finish defining the linkedin_small_real.json schema
# We've provided most of the fiddly details, but you'll
# need to fill in **name**, ** experience ** and **events** fields!

schema = StructType([
    StructField("_id", StringType(), nullable=True),
    
    StructField("education", ArrayType(
        StructType([
          StructField("start", StringType(), nullable=True),
          StructField("major", StringType(), nullable=True),
          StructField("end", StringType(), nullable=True),
          StructField("name", StringType(), nullable=True),
          StructField("degree", StringType(), nullable=True),
          StructField("desc", StringType(), nullable=True)
    ])), nullable=True), 

    StructField("group", StructType([
          StructField("affilition", ArrayType(StringType()), nullable=True),
          StructField("member", StringType(), nullable=True)
    ]), nullable=True), 

    StructField("locality", StringType(), nullable=True),
    StructField("skills", ArrayType(StringType()), nullable=True),
    StructField("industry", StringType(), nullable=True),
    StructField("interval", IntegerType(), nullable=True),

    StructField("summary", StringType(), nullable=True),
    StructField("interests", StringType(), nullable=True),
    StructField("overview_html", StringType(), nullable=True),
    StructField("specilities", StringType(), nullable=True),
    StructField("homepage", ArrayType(StringType()), nullable=True),
    StructField("honors", ArrayType(StringType()), nullable=True),
    StructField("url", StringType(), nullable=True),
    StructField("also_view", ArrayType(
      StructType([
          StructField("id", StringType(), nullable=True),
          StructField("url", StringType(), nullable=True)
      ])
    ), nullable=True),

    
    # TODO: fill in the necessary structure for name 

    StructField("name", StructType([
          StructField("family_name", StringType(), nullable=True),
          StructField("given_name", StringType(), nullable=True)
    ]), nullable=True),

    # TODO: fill in the necessary structure for experience

    StructField("experience", ArrayType(StructType([
          StructField("org", StringType(), nullable=True),
          StructField("title", StringType(), nullable=True),
          StructField("end", StringType(), nullable=True),
          StructField("start", StringType(), nullable=True),
          StructField("desc", StringType(), nullable=True)
    ])), nullable=True),


    # TODO: fill in the necessary structure for events 

    StructField("events", ArrayType(StructType([
          StructField("from", StringType(), nullable=True),
          StructField("to", StringType(), nullable=True),
          StructField("title1", StringType(), nullable=True),
          StructField("start", IntegerType(), nullable=True),
          StructField("title2", StringType(), nullable=True),
          StructField("end", IntegerType(), nullable=True)
    ])), nullable=True)
    
  
])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 1.2: The Langorous Load [4 Pts]

#### 1.2.1: Load LinkedIn Dataset [4 Pts]

In the following cell, you will load the `linkedin_small_real.json` dataset from your S3 bucket into a Spark dataframe (sdf) called `linkedin_data_sdf`. If you have constructed `schema` correctly then `spark.read.json()` will read in the dataset. ***You do not need to edit this cell***.

If this doesn't work, go back to the prior cell and update your `schema`!

Note that the cell below will load data even if your schema is incomplete and has left out some columns of the data, so be sure to check that you have included all of the fields from the JSON.

In [ ]:
%%spark

linkedin_data_sdf = spark.read.json("s3a://penn-cis545-files/linkedin_small_real.json", schema=schema)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark
# Let's print out the first few rows to see how the data looks like in tabular form
linkedin_data_sdf.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------+------+--------------------+--------------------+--------------------+--------------------+--------------------+
|             _id|           education|               group|            locality|              skills|            industry|interval|             summary|           interests|       overview_html|         specilities|homepage|honors|                 url|           also_view|                name|          experience|              events|
+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------+------+--------------------+--------------------+--------------------+--------------------+--------

In [ ]:
import pandas as pd

The cell below shows how to run SQL commands on Spark tables. **Use this as a template for all your SQL queries in this notebook.** 

For almost all the questions you will need to create a temporary view using `createOrReplaceTempView`, then write your sql `query` as a string and then run the query on spark using `spark.sql(query)`. To see what your query resulted use `.show()`.

***You do not need to edit this cell***.

In [ ]:
%%spark


# Create SQL-accesible table
linkedin_data_sdf.createOrReplaceTempView("linkedin_data")

# Declare SQL query to be excecuted
query = '''SELECT * 
           FROM linkedin_data 
           ORDER BY _id
           LIMIT 10'''

# Save the output sdf of spark.sql() as answer_sdf
answer_sdf = spark.sql(query)
answer_sdf.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------+------+--------------------+--------------------+--------------------+--------------------+--------------------+
|               _id|           education|               group|            locality|              skills|            industry|interval|             summary|           interests|       overview_html|         specilities|homepage|honors|                 url|           also_view|                name|          experience|              events|
+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------+------+--------------------+--------------------+--------------------+--------------------+--

We can then copy the `answer_sdf` to Colab to submit to PennGrader...

In [ ]:
#Convert to Pandas
%spark -o answer_sdf

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
## AUTOGRADER 
grader.grade(test_case_id = 'test_schema_linkedin', answer = answer_sdf)

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.


#### 1.2.2: SQL refresher [3 Pts]


In the next cell, we want you to create `industry_family_name_df` to fetch the data from the `linkedin_data` table created above, returning rows with schema `(_id, industry, family_name)`. Remove all NULLs from the `family_name` and `industry` columns. Sort the columns by `_id, industry, family_name`, all ascending order.  Limit your sdf to 100 rows.

Hint: To access a field inside a json object in SQL you can use the following syntax: `parent_variable.child_variable`

In [ ]:
%%spark 

#TODO

# Declare SQL query to be excecuted

query = '''SELECT _id, industry, name.family_name
           FROM linkedin_data 
           WHERE industry is not null and name.family_name is not null
           ORDER BY _id, industry, name.family_name
           LIMIT 100'''

industry_family_name_df = spark.sql(query)
industry_family_name_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------+-------------+
|                _id|            industry|  family_name|
+-------------------+--------------------+-------------+
|        in-00000001|     Medical Devices|   Mazalu MBA|
|           in-00001|     Pharmaceuticals|     Forslund|
|           in-00006|            Research|      Douglas|
|   in-000montgomery|Information Techn...|     Kilimann|
| in-000vijaychauhan|Aviation & Aerospace| Chauhan, PMP|
|   in-001adambutler|Marketing and Adv...|         Adam|
|       in-001monica|Nonprofit Organiz...|       Andrus|
|  in-001neilpeacock|Computer & Networ...|      Peacock|
|           in-00666|    Telekomünikasyon|BOLUKBAS, PMP|
|        in-00789123|  Telecommunications|     Hoffmann|
|in-007katarzynagola|Zarządzanie dział...|         Gola|
|           in-00911|     Human Resources|      Smekens|
|in-00danielnewman00|   Computer Software|       Newman|
|   in-00jenmiller00|            Internet|       Miller|
|in-00stevensaunders|          

In [ ]:
#Convert to Pandas
%spark -o industry_family_name_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
## AUTOGRADER 
grader.grade(test_case_id = 'test_industry_family_name_df', answer = industry_family_name_df)

Correct! You earned 3/3 points. You are a star!

Your submission has been successfully recorded in the gradebook.


#### 1.2.3: Load Stock Prices Data [3 Pts]

Just the way you created a schema for the LinkedIn dataset, now create a schema for the Stock Prices data. The schema should be relatively simple, compared to the LinkedIn schema. A tiny version of the data is [here](https://docs.google.com/spreadsheets/d/1TStiS-bwkCJR1w5rJ18QPlNe3SIK2Z8QS9gK6ltnjJQ/edit?usp=sharing) in csv format, so you can see what the types should be for the different fields (columns in the csv). Store the `Date` field as a String.


In [ ]:
%%spark

from pyspark.sql.types import *

# TODO

stocks_schema = StructType([
    StructField("Date", StringType(), nullable=True),
    StructField("Open", FloatType(), nullable=True),
    StructField("High", FloatType(), nullable=True),
    StructField("Low", FloatType(), nullable=True),
    StructField("Close", FloatType(), nullable=True),
    StructField("Volume", IntegerType(), nullable=True),
    StructField("OpenInt", IntegerType(), nullable=True),
    StructField("org", StringType(), nullable=True)
])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In the following cell, you will load the entire `stocks.csv` dataset from your S3 bucket into a Spark dataframe (sdf) called `stocks_sdf`. If you have constructed `schema` correctly then `spark.read.format("csv")` will read in the dataset. **You do not need to edit this cell.**

If this doesn't work, go back to the prior cell and update your schema!

In [ ]:
#Do not edit
%%spark

# Load stock data

stocks_sdf = spark.read.format("csv") \
              .option("header", "true") \
              .schema(stocks_schema) \
              .load("s3a://penn-cis545-files/stocks.csv")

# Creates SQL-accesible table
stocks_sdf.createOrReplaceTempView('stocks')

# Display the first 10 rows
query = '''SELECT *
           FROM stocks'''
answer_stocks_sdf = spark.sql(query)
answer_stocks_sdf.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+------+------+------+--------+-------+---+
|      Date|  Open|  High|   Low| Close|  Volume|OpenInt|org|
+----------+------+------+------+------+--------+-------+---+
|1999-11-18|30.713|33.754|27.002|29.702|66277506|      0|  A|
|1999-11-19|28.986|29.027|26.872|27.257|16142920|      0|  A|
|1999-11-22|27.886|29.702|27.044|29.702| 6970266|      0|  A|
|1999-11-23|28.688|29.446|27.002|27.002| 6332082|      0|  A|
|1999-11-24|27.083|28.309|27.002|27.717| 5132147|      0|  A|
|1999-11-26|27.594|28.012|27.509|27.807| 1832635|      0|  A|
|1999-11-29|27.676| 28.65| 27.38|28.432| 4317826|      0|  A|
|1999-11-30| 28.35|28.986|27.634| 28.48| 4567146|      0|  A|
|1999-12-01| 28.48|29.324|28.273|28.986| 3133746|      0|  A|
|1999-12-02|29.532|30.375|29.155|29.786| 3252997|      0|  A|
+----------+------+------+------+------+--------+-------+---+
only showing top 10 rows

In [ ]:
#Convert to Pandas
%spark -o answer_stocks_sdf

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
## AUTOGRADER 
grader.grade(test_case_id = 'test_schema_stocks', answer = answer_stocks_sdf)

Correct! You earned 3/3 points. You are a star!

Your submission has been successfully recorded in the gradebook.


#### 1.2.4: Calculate Percentage Change [4 Pts]

In the next cell, we want you to display the percentage change in the daily stock prices for each organization. In order to do so, we will need the data from the `stocks_sdf` table created above. Create a new column called `percentage_change` that uses the opening and closing stock prices for each organization, for each day, and calculates the percentage change in the stock price as follows: 

\begin{align}
percentage\_change = \frac{close-open}{open}*100.0
\end{align}

In order to avoid nulls, calculate the percentage change for only for those organizations and days where the **opening price is NOT 0.0**. The percentage_change value is a float.

Your final dataframe should include all columns of the original `stocks_sdf`, as well as the new column you create called `percentage_change`. Sort the rows by **Date and org** ascending, in that order.

In [ ]:
%%spark


# Declare SQL query to be excecuted
query = '''SELECT *, (Close-Open)/Open*100.0 as percentage_change
           FROM stocks 
           WHERE Open <> 0.0
           ORDER BY Date, org'''

# Save the output sdf of spark.sql() as answer_sdf
answer_sdf = spark.sql(query)
answer_sdf.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+------+------+------+-------+-------+---+-------------------+
|      Date|  Open|  High|   Low| Close| Volume|OpenInt|org|  percentage_change|
+----------+------+------+------+------+-------+-------+---+-------------------+
|1962-01-02|0.6277|0.6362|0.6201|0.6201|2575579|      0| GE|-1.2107615398186964|
|1962-01-02| 6.413| 6.413|6.3378|6.3378| 467056|      0|IBM|-1.1726193609712994|
|1962-01-03|0.6201|0.6201|0.6122|0.6201|1764749|      0| GE|                0.0|
|1962-01-03|6.3378|6.3963|6.3378|6.3963| 350294|      0|IBM| 0.9230302761273703|
|1962-01-04|0.6201|0.6201|0.6037|0.6122|2194010|      0| GE|-1.2739879610348466|
|1962-01-04|6.3963|6.3963|6.3295|6.3295| 314365|      0|IBM|-1.0443481752999644|
|1962-01-05|0.6122|0.6122|0.5798|0.5957|3255244|      0| GE|-2.6951969289881075|
|1962-01-05|6.3211|6.3211|6.1958|6.2041| 440112|      0|IBM| -1.850945225473035|
|1962-01-08|0.5957|0.5957|0.5716|0.5957|3696430|      0| GE|                0.0|
|1962-01-08|6.2041|6.2041|6.

In [ ]:
#################     DO NOT EDIT      ##################
%%spark
answer_sdf.createOrReplaceTempView("test_1_2_4")
test_1_2_4_sdf = spark.sql("SELECT * FROM test_1_2_4 LIMIT 10")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#Convert to Pandas
%spark -o test_1_2_4_sdf

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
## AUTOGRADER 
grader.grade(test_case_id = 'test_perc_change', answer = test_1_2_4_sdf)

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.


### 1.3: Cleaning LinkedIn Data

#### 1.3.1: Adding Experience [6 Pts]

In this part, we are interested in when individuals began working at a particular company.  When you created the schema, you might have noticed that the collection of companies an invidual worked at are contained in the `experience` field as an **array of dictionaries**. The company name is stored in the `org` field, and the start date is in the `start` field. Here is an example of an `experience` field:

```
{
   "experience": [
     {
        "org": "The Walt Disney Company", 
        "title" : "Mickey Mouse",
        "end" : "Present",
        "start": "November 1928",
        "desc": "Sailed a boat."
     },
     {
        "org": "Walt Disney World Resort",
        "title": "Mickey Mouse Mascot",
        "start": "January 2005",
        "desc": "Took pictures with kids."
     }
   ]
}
```

Your task is to extract each pair of company and start date from these arrays. This is known as "exploding" a row in Spark. If you think about how we used relational data to model a nested list in a separate table -- that's basically what an explode does to the nested data within `linkedin`.

Create an sdf called `raw_start_dates_sdf` that contains the company and start date for every experience of every individual in `linkedin_data_sdf`. Drop any row that contains a `null` in either column. 

```
+--------------------------+---------------+
|org                       |start_date     |
+--------------------------+---------------+
|Walt Disney World Resort  |January 2005   | 
|The Walt Disney Company   |November 1928  |
|...                       |...            |
+--------------------------+---------------+
```

_Hint_: Reference the [function list](https://spark.apache.org/docs/2.3.0/api/sql/index.html).

_Note_: Some of the entires in `org` are "weird", i.e. made up of non-english letters and characters. Keep them. **DO NOT** edit any name in the original dataframe unless we specify. **DO NOT** drop any row unless there is a `null` value as stated before. This goes for the rest of the homework as well, unless otherwise specified.

In [ ]:
%%spark 

#ToDo

# Declare SQL query to be excecuted
query = '''SELECT exp.org, exp.start as start_date
           FROM (
           SELECT explode(experience) as exp
           FROM linkedin_data)
           WHERE exp.org is not null
           AND exp.start is not null
           '''

raw_start_dates_sdf = spark.sql(query)
raw_start_dates_sdf.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------+
|                 org|    start_date|
+--------------------+--------------+
| Johnson and Johnson| November 2009|
|Albert Einstein M...|September 2008|
| Columbia University|   August 2006|
|Memorial Sloan Ke...|  January 2003|
|Sahlgrenska Unive...| November 2001|
|                UCSF|September 2012|
|Wyss Institute fo...|      May 2009|
|<Online Recruitin...| December 2009|
|<Medical Testing ...| December 2009|
|   000Montgomery.Com|  January 2005|
|<Software Consult...|   August 2000|
|<Software Consult...| December 2008|
|<Temperature Sens...| February 2009|
|<Advertising Comp...|      May 2007|
|<Advertising Comp...|    March 2007|
|<Psychology Organ...|  January 2007|
|<Advertising Comp...| February 2007|
|<National Recruit...| February 2006|
| <Employment Agency>|  October 2005|
|<Biometrics Hardw...|     July 2006|
+--------------------+--------------+
only showing top 20 rows

In [ ]:
#################     DO NOT EDIT      ##################
%%spark
raw_start_dates_sdf.createOrReplaceTempView("test_1_3_1")
test_1_3_1_sdf = spark.sql("SELECT * FROM test_1_3_1 ORDER BY org ASC, start_date DESC LIMIT 20")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#Convert to Pandas
%spark -o test_1_3_1_sdf

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
## AUTOGRADER 
grader.grade(test_case_id = 'test_pos_explode', answer = test_1_3_1_sdf)

Correct! You earned 6/6 points. You are a star!

Your submission has been successfully recorded in the gradebook.


#### 1.3.2: Filtering on Date [5 Pts]

There are two issues with the values in our `date` column. First, the values are saved as strings, not datetime types. This halts us from running functions such as `ORDER BY` or `GROUP BY` on common months or years. Second, some values do not have both month and year information or are in other languages. Your task is to filter out and clean the `date` column. We are interested in only those rows that have date in the following format `"(month_name) (year)"`, e.g. "October 2010".

Using `raw_start_dates_sdf`, create an sdf called `filtered_start_dates_sdf` with the `date` column filtered in the manner above. **Keep only those rows with a start date between January 2000 ('2000-01-01') to December 2011 ('2011-12-01'), inclusive**.  Ensure that any dates that are not in our desired format are omitted. Drop any row that contains a `null` in either column. The format of the sdf is shown below:
```
+--------------------------+---------------+
|org                       |start_date     |
+--------------------------+---------------+
|Walt Disney World Resort  |2005-01-01     | 
|...                       |...            |
+--------------------------+---------------+
```
_Hint_: Refer to the [function list](https://spark.apache.org/docs/2.3.0/api/sql/index.html) to format the `date` column. In Spark SQL the date format we are interested in is `"MMM y"`.

_Note_: Spark will return the date in the format above, with the day as `01`. This is ok, since we are interested in the month and year each individual began working and all dates will have `01` as their day.

In [ ]:
%%spark

#Do not remove the line below
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

#ToDo
query = '''SELECT org, start_date
           FROM
           (SELECT org, TO_DATE(start_date,'MMM y') as start_date
           FROM test_1_3_1)
           WHERE start_date <= '2011-12-01'
           AND start_date >= '2000-01-01'
           '''

filtered_start_dates_sdf = spark.sql(query)
filtered_start_dates_sdf.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+
|                 org|start_date|
+--------------------+----------+
| Johnson and Johnson|2009-11-01|
|Albert Einstein M...|2008-09-01|
| Columbia University|2006-08-01|
|Memorial Sloan Ke...|2003-01-01|
|Sahlgrenska Unive...|2001-11-01|
|Wyss Institute fo...|2009-05-01|
|<Online Recruitin...|2009-12-01|
|<Medical Testing ...|2009-12-01|
|   000Montgomery.Com|2005-01-01|
|<Software Consult...|2000-08-01|
|<Software Consult...|2008-12-01|
|<Temperature Sens...|2009-02-01|
|<Advertising Comp...|2007-05-01|
|<Advertising Comp...|2007-03-01|
|<Psychology Organ...|2007-01-01|
|<Advertising Comp...|2007-02-01|
|<National Recruit...|2006-02-01|
| <Employment Agency>|2005-10-01|
|<Biometrics Hardw...|2006-07-01|
|<Product Manufact...|2001-08-01|
+--------------------+----------+
only showing top 20 rows

In [ ]:
#################     DO NOT EDIT      ##################
%%spark
filtered_start_dates_sdf.createOrReplaceTempView("test_1_3_2")
test_1_3_2_sdf = spark.sql("SELECT * FROM ((SELECT org, DATE_FORMAT(start_date, 'yyyy-MM-dd') AS start_date FROM test_1_3_2 ORDER BY start_date DESC, org DESC LIMIT 10) UNION (SELECT org, DATE_FORMAT(start_date, 'yyyy-MM-dd') AS start_date FROM test_1_3_2 ORDER BY start_date ASC, org ASC LIMIT 10)) ORDER BY start_date ASC, org ASC")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#Convert to Pandas
%spark -o test_1_3_2_sdf

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
## AUTOGRADER 
grader.grade(test_case_id = 'test_linkedin_date', answer = test_1_3_2_sdf)

Correct! You earned 5/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.


### 1.4: Cleaning Stock Data

#### 1.4.1: Adding Company Names [2 Pts]

In part 4 of this homework, we have to merge the stocks and linkedin dataframes. This would be difficult to do directly, as the companies in our stock dataset are defined by their stock tickers instead of the full names. Thus, we would not be able to merge it with the `org` field in `hire_train_sdf`. We must convert them to that format. For this purpose, we can create a user-defined function (udf) to achieve the mentioned conversion.

A udf is defined as a normal Python function and then registered to be used as a Spark SQL function. Your task is to create a udf, `TICKER_TO_NAME()` that will convert the ticker field in `raw_stocks` to the company's name. This will be done using the provided `ticker_to_name_dict` dictionary. We are only interested in the companies in that dictionary.

Fill out the function `ticker_to_name()` below. Then use `spark.udf.register()` to register it as a SQL function. The command is provided. ***You do not need to edit it***. Note, we have defined the udf as returning `StringType()`. Ensure that your function returns this. You must also deal with any potential `null` cases.

If the ticker value isn't in the table, set it to a string equal to `None`.

In [ ]:
%%spark

# Dictionary linking stock ticker symbols to their names
ticker_to_name_dict = {'NOK': 'Nokia',
                       'UN': 'Unilever',
                       'BP': 'BP',
                       'JNJ': 'Johnson & Johnson',
                       'TCS': 'Tata Consultancy Services',
                       'SLB': 'Schlumberger',
                       'NVS': 'Novartis',
                       'CNY': 'Huawei',
                       'PFE': 'Pfizer',
                       'ACN': 'Accenture',
                       'DELL': 'Dell',
                       'MS': 'Morgan Stanley',
                       'ORCL': 'Oracle',
                       'BAC': 'Bank of America',
                       'PG': 'Procter & Gamble',
                       'CGEMY': 'Capgemini',
                       'GS': 'Goldman Sachs',
                       'C': 'Citi',
                       'IBM': 'IBM',
                       'CS': 'Credit Suisse',
                       'MDLZ': 'Kraft Foods',
                       'WIT': 'Wipro Technologies',
                       'CSCO': 'Cisco Systems',
                       'PWC': 'PwC',
                       'GOOGL': 'Google',
                       'CTSH': 'Cognizant Technology Solutions',
                       'HSBC': 'HSBC',
                       'DB': 'Deutsche Bank',
                       'MSFT': 'Microsoft',
                       'HPE': 'Hewlett-Packard',
                       'ERIC': 'Ericsson',
                       'BCS': 'Barclays Capital',
                       'GSK': 'GlaxoSmithKline'}


# TODO: Fill out [ticker_to_name()] and register it as a udf.


def ticker_to_name(ticker):
    #TODO
    if ticker in ticker_to_name_dict.keys():
      return ticker_to_name_dict[ticker]
    else:
      return None

  

# Register udf as a SQL function. DO NOT EDIT
spark.udf.register("TICKER_TO_NAME", ticker_to_name, StringType())


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<function ticker_to_name at 0x7f2605b193b0>

In [ ]:
#################     DO NOT EDIT      ##################
%%spark

ticker_to_name = [((str(ticker_to_name("GOOGL")),str(ticker_to_name("TSLA"))))]
columns = ['A', 'B']
dataframe = spark.createDataFrame(ticker_to_name, columns)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#Convert to Pandas
%spark -o dataframe

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
## AUTOGRADER 
grader.grade(test_case_id = 'test_ticker_name', answer = dataframe)

Correct! You earned 2/2 points. You are a star!

Your submission has been successfully recorded in the gradebook.


#### 1.4.2: Wrangling stocks data [5 Pts]

We can now begin to wrangle `stocks_sdf` with our new `TICKER_TO_NAME()` function.

Create an sdf called `filter_1_stocks_sdf` as follows. Convert all the ticker names in `stocks_sdf` to the company names and save it as `org`. Next, convert the `date` field to a datetime type. As explained before this will help order and group the rows in future steps. 

Drop any company names that do not appear in `ticker_to_name_dict`. **Keep any date between January 1st 2001 ('2001-01-01') and December 4th 2012 ('2012-12-04') inclusive**, in the format shown below (note this is a datetime object not a string):

```
+----+------------+--------------+
|org |date        |Close         |
+----+------------+--------------+
|IBM |2000-01-03  |...           |
|... |...         |...           |
+----+------------+--------------+
```
_Hint_: You will use a similar function to filter the dates as in Step 1.3.2. In Spark SQL the format for the `date` field in `raw_stocks_sdf` is `"yyyy-MM-dd"`.

In [ ]:
%%spark

# TODO
query = '''
           SELECT Date, org, Close
           FROM
           (SELECT to_date(Date,'yyyy-MM-dd') as Date, ticker_to_name(org) as org, Close
           FROM stocks
           WHERE Date <= '2012-12-04'
           AND Date >= '2001-01-01')
           WHERE org is not null
           '''

filter_1_stocks_sdf = spark.sql(query)
filter_1_stocks_sdf.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------+------+
|      Date|      org| Close|
+----------+---------+------+
|2005-02-25|Accenture|21.363|
|2005-02-28|Accenture|21.704|
|2005-03-01|Accenture|21.735|
|2005-03-02|Accenture|21.593|
|2005-03-03|Accenture|21.329|
|2005-03-04|Accenture|21.549|
|2005-03-07|Accenture|21.651|
|2005-03-08|Accenture|21.509|
|2005-03-09|Accenture|21.379|
|2005-03-10|Accenture|21.194|
|2005-03-11|Accenture|20.777|
|2005-03-14|Accenture|20.794|
|2005-03-15|Accenture| 20.37|
|2005-03-16|Accenture| 20.41|
|2005-03-17|Accenture|20.344|
|2005-03-18|Accenture|20.098|
|2005-03-21|Accenture|20.395|
|2005-03-22|Accenture|20.623|
|2005-03-23|Accenture|20.641|
|2005-03-24|Accenture|20.726|
+----------+---------+------+
only showing top 20 rows

In [ ]:
#################     DO NOT EDIT      ##################
%%spark
filter_1_stocks_sdf.createOrReplaceTempView("test_1_4_2")
test_1_4_2_sdf = spark.sql("SELECT * FROM ((SELECT org, DATE_FORMAT(date, 'yyyy-MM-dd') as date, Close FROM test_1_4_2 ORDER BY date DESC, org DESC LIMIT 10) UNION (SELECT org, DATE_FORMAT(date, 'yyyy-MM-dd') as date, Close FROM test_1_4_2 ORDER BY date ASC, org ASC LIMIT 10)) ORDER BY date ASC, org DESC")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#Convert to Pandas
%spark -o test_1_4_2_sdf

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
## AUTOGRADER 
grader.grade(test_case_id = 'test_stock_date', answer = test_1_4_2_sdf)

Correct! You earned 5/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## Step 2: Analysis on LinkedIn Data [12 pts]

### 2.1: Counting Employees [2 Pts]

Now we would like to find for each company, the number of individuals who started in the same month and year. Use `filtered_start_dates_sdf` and create a new sdf called `start_dates_sdf` which will contain the total number of employees who began working at the same company on the same start date (name the new column as `num_employees`). The format of the sdf is shown below:

```
+--------------------------+---------------+---------------+
|org                       |start_date     |num_employees  |
+--------------------------+---------------+---------------+
|Walt Disney World Resort  |2005-01-01     |1              |
|...                       |...            |...            |
+--------------------------+---------------+---------------+
```

In [ ]:
%%spark

# TODO: 

query = """
            SELECT org, start_date, count(org) as num_employees
            FROM test_1_3_2
            GROUP BY org, start_date
"""

start_dates_sdf = spark.sql(query)
start_dates_sdf.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+-------------+
|                 org|start_date|num_employees|
+--------------------+----------+-------------+
|     Electronic Arts|2006-01-01|            2|
|              Toyota|2004-07-01|            1|
|  Financiële Fabriek|2002-02-01|            1|
|         Eatops Sarl|2008-05-01|            1|
|     Barry Callebaut|2006-09-01|            1|
|       Berkvens B.V.|2006-06-01|            1|
|         Trend Micro|2011-11-01|            1|
|            mCentric|2000-08-01|            1|
| Livingstone Estates|2006-07-01|            1|
|Georgia Institute...|2008-08-01|            6|
|      Embassy Suites|2000-05-01|            1|
|         Au Bon Pain|2007-12-01|            1|
|        Celtel Congo|2004-02-01|            1|
|           JP Morgan|2007-08-01|            1|
|           95.5 WBRU|2004-04-01|            1|
|                 UPS|2004-09-01|            1|
|            T-Mobile|2002-05-01|            1|
|             Cadbury|2010-01-01|       

In [ ]:
#################     DO NOT EDIT      ##################
%%spark

start_dates_sdf.createOrReplaceTempView("test_2_1")
test_2_1_sdf = spark.sql("SELECT org, DATE_FORMAT(start_date, 'yyyy-MM-dd') as start_date, num_employees FROM test_2_1 ORDER BY num_employees DESC, org DESC, start_date ASC LIMIT 10")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#Convert to Pandas
%spark -o test_2_1_sdf

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
## AUTOGRADER 
grader.grade(test_case_id = 'test_num_employees', answer = test_2_1_sdf)

Correct! You earned 2/2 points. You are a star!

Your submission has been successfully recorded in the gradebook.


### 2.2: Reshape DataFrame [7 Pts]

Our next step is to use `start_dates_sdf` and create a new sdf called `raw_hire_train_sdf` that has for a single company and a single year, the number of hires in Jan through Dec, as well as the total number of hires that year (name it `total_num`). Note that for each company you will have several rows corresponding to years between 2000 and 2011. It is alright if for a given company you don't have a given year. However, ensure that for a given company and given year, each month column has an entry, i.e. if no one was hired the value should be `0`.  

_Note_: We will use the first three letters of each month in naming, i.e. `jan, feb, mar, apr, may, jun, jul, aug, sep, oct, nov, dec`.

The format of the `raw_hire_train_sdf` is shown below:

```
+----+-----+----------+---------+----------+----------+
|org |year |jan_hired |   ...   |dec_hired |total_num |
+----+-----+----------+---------+----------+----------+
|IBM |2008 |...       |   ...   |...       |...       |
|IBM |2009 |...       |   ...   |...       |...       |
|... |...  |...       |   ...   |...       |...       |
+----+-----+----------+---------+----------+----------+
```


In [ ]:
%%spark

# TODO:

query = """ SELECT org, 
                   year, 
                   sum(case when month = 1 then num_employees else 0 end) as jan_hired,
                   sum(case when month = 2 then num_employees else 0 end) as feb_hired,
                   sum(case when month = 3 then num_employees else 0 end) as mar_hired,
                   sum(case when month = 4 then num_employees else 0 end) as apr_hired,
                   sum(case when month = 5 then num_employees else 0 end) as may_hired,
                   sum(case when month = 6 then num_employees else 0 end) as jun_hired,
                   sum(case when month = 7 then num_employees else 0 end) as jul_hired,
                   sum(case when month = 8 then num_employees else 0 end) as aug_hired,
                   sum(case when month = 9 then num_employees else 0 end) as sep_hired,
                   sum(case when month = 10 then num_employees else 0 end) as oct_hired,
                   sum(case when month = 11 then num_employees else 0 end) as nov_hired,
                   sum(case when month = 12 then num_employees else 0 end) as dec_hired,
                   sum(num_employees) as total_num

            FROM 
            (SELECT org, num_employees, date_format(start_date,'Y') as year, date_format(start_date,'M') as month
            FROM test_2_1)
            GROUP BY org, year
"""

raw_hire_train_sdf = spark.sql(query)
raw_hire_train_sdf.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|                 org|year|jan_hired|feb_hired|mar_hired|apr_hired|may_hired|jun_hired|jul_hired|aug_hired|sep_hired|oct_hired|nov_hired|dec_hired|total_num|
+--------------------+----+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  Hersha Hospitality|2011|        0|        0|        0|        0|        0|        0|        0|        0|        0|        1|        0|        0|        1|
|U.S. Green Buildi...|2008|        0|        0|        0|        1|        0|        0|        0|        0|        0|        0|        0|        0|        1|
|  Home Dynamics Corp|2007|        0|        0|        0|        0|        0|        0|        0|        0|        0|        0|        1|        0|        1|
|Commonwealth Edit...|2002|        0|        0|     

In [ ]:
#################     DO NOT EDIT      ##################
%%spark
raw_hire_train_sdf.createOrReplaceTempView("test_2_2")
test_2_2_sdf = spark.sql("SELECT * FROM test_2_2 ORDER BY total_num DESC, org DESC, year ASC LIMIT 20")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#Convert to Pandas
%spark -o test_2_2_sdf

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
## AUTOGRADER 
grader.grade(test_case_id = 'test_full_hired', answer = test_2_2_sdf)

Correct! You earned 7/7 points. You are a star!

Your submission has been successfully recorded in the gradebook.


### 2.3: Filtering on Company Size [3 Pts]

Create an sdf called `hire_train_sdf` that contains all the observations in `raw_hire_train_sdf` with `total_num` greater than or equal to 20. 

In [ ]:
%%spark

# TODO
# Keep all rows where total_num >= 20

query = """ SELECT *
            FROM test_2_2
            WHERE total_num >= 20
"""

hire_train_sdf = spark.sql(query)
hire_train_sdf.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|                 org|year|jan_hired|feb_hired|mar_hired|apr_hired|may_hired|jun_hired|jul_hired|aug_hired|sep_hired|oct_hired|nov_hired|dec_hired|total_num|
+--------------------+----+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|HP Enterprise Ser...|2011|        3|        0|        3|        1|        1|        2|        2|        5|        2|        1|        1|        0|       21|
|Tata Consultancy ...|2006|        4|        2|        1|        2|        1|        2|        1|        2|        5|        2|        2|        2|       26|
|            Ericsson|2010|        1|        1|        0|        3|        1|        4|        6|        5|        5|        1|        2|        2|       31|
|       Cisco Systems|2010|        4|        6|     

In [ ]:
#################     DO NOT EDIT      ##################
%%spark
hire_train_sdf.createOrReplaceTempView("test_2_3")
test_2_3_sdf = spark.sql("SELECT * FROM test_2_3 ORDER BY org ASC, year ASC LIMIT 10")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#Convert to Pandas
%spark -o test_2_3_sdf

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
## AUTOGRADER 
grader.grade(test_case_id = 'test_sampled_hired', answer = test_2_3_sdf)

Correct! You earned 3/3 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## Step 3: Analyzing Stock Data [19 pts]

### 3.1: Average Closing Price [5 Pts]

The data in `filter_1_stocks_sdf` gives closing prices on a daily basis. Since we are interested in monthly trends, we will only keep the **average of the closing price of each month per year for each org**. 

Create an sdf `filter_2_stocks_sdf` that contains only the average of closing prices for each month-year pair sorted by the org alphabetically and then month, year from earliest to latest with the closing price rounded off to 3 decimal places. The format of the sdf is shown below:

```
+----+------------+--------------+--------------+
|org |month       |year          |close         |
+----+------------+--------------+--------------+
|IBM |01          |2000          |...           |
|... |...         |...           |...           |
+----+------------+--------------+--------------+
```

In [ ]:
%%spark

query='''
      SELECT org, CAST(date_format(date,'M') AS int) AS month, CAST(date_format(date,'y')AS int) AS year, ROUND(AVG(Close),3) AS close
      FROM test_1_4_2
      GROUP BY org, month, year
      ORDER BY org, year, month
      '''

filter_2_stocks_sdf = spark.sql(query)
filter_2_stocks_sdf.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-----+----+------+
|      org|month|year| close|
+---------+-----+----+------+
|Accenture|    2|2005|21.534|
|Accenture|    3|2005|20.904|
|Accenture|    4|2005|19.467|
|Accenture|    5|2005|18.977|
|Accenture|    6|2005|19.892|
|Accenture|    7|2005|20.492|
|Accenture|    8|2005|21.238|
|Accenture|    9|2005|21.629|
|Accenture|   10|2005|21.897|
|Accenture|   11|2005|23.048|
|Accenture|   12|2005|24.367|
|Accenture|    1|2006|26.098|
|Accenture|    2|2006|27.378|
|Accenture|    3|2006|26.787|
|Accenture|    4|2006|25.187|
|Accenture|    5|2006|24.343|
|Accenture|    6|2006|22.946|
|Accenture|    7|2006|24.066|
|Accenture|    8|2006|24.245|
|Accenture|    9|2006|24.995|
+---------+-----+----+------+
only showing top 20 rows

In [ ]:
#################     DO NOT EDIT      ##################
%%spark
filter_2_stocks_sdf.createOrReplaceTempView("test_3_1")
test_3_1_sdf = spark.sql("SELECT * FROM test_3_1 LIMIT 10")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#Convert to Pandas
%spark -o test_3_1_sdf

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
## AUTOGRADER 
grader.grade(test_case_id = 'test_summary', answer = test_3_1_sdf)

Correct! You earned 5/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.


### 3.2: Reshape DataFrame Again! [8 Pts]

Now, we will begin to shape our dataframe into the format of the final training sdf.

Create an sdf `filter_3_stocks_sdf` that has for a single company and a single year, the average stock price for each month in that year. This is similar to the table you created in Step 3.1. If the data is not avaliable, drop any rows containing any `null` values, in any column. The format of the sdf is shown below:

```
+----+-----+----------+---------+----------+
|org |year |jan_stock |   ...   |dec_stock |
+----+-----+----------+---------+----------+
|IBM |2008 |...       |   ...   |...       |
|IBM |2009 |...       |   ...   |...       |
|... |...  |...       |   ...   |...       |
+----+-----+----------+---------+----------+
```


In [ ]:
%%spark

#ToDo

query = """ SELECT org, 
                   year, 
                   if(sum(case when month = 1 then 1 else 0 end) <> 0, sum(case when month = 1 then close else 0 end)/sum(case when month = 1 then 1 else 0 end), 0) as jan_stock,
                   if(sum(case when month = 2 then 1 else 0 end) <> 0, sum(case when month = 2 then close else 0 end)/sum(case when month = 2 then 1 else 0 end), 0) as feb_stock,
                   if(sum(case when month = 3 then 1 else 0 end) <> 0, sum(case when month = 3 then close else 0 end)/sum(case when month = 3 then 1 else 0 end), 0) as mar_stock,
                   if(sum(case when month = 4 then 1 else 0 end) <> 0, sum(case when month = 4 then close else 0 end)/sum(case when month = 4 then 1 else 0 end), 0) as apr_stock,
                   if(sum(case when month = 5 then 1 else 0 end) <> 0, sum(case when month = 5 then close else 0 end)/sum(case when month = 5 then 1 else 0 end), 0) as may_stock,
                   if(sum(case when month = 6 then 1 else 0 end) <> 0, sum(case when month = 6 then close else 0 end)/sum(case when month = 6 then 1 else 0 end), 0) as jun_stock,
                   if(sum(case when month = 7 then 1 else 0 end) <> 0, sum(case when month = 7 then close else 0 end)/sum(case when month = 7 then 1 else 0 end), 0) as jul_stock,
                   if(sum(case when month = 8 then 1 else 0 end) <> 0, sum(case when month = 8 then close else 0 end)/sum(case when month = 8 then 1 else 0 end), 0) as aug_stock,
                   if(sum(case when month = 9 then 1 else 0 end) <> 0, sum(case when month = 9 then close else 0 end)/sum(case when month = 9 then 1 else 0 end), 0) as sep_stock,
                   if(sum(case when month = 10 then 1 else 0 end) <> 0, sum(case when month = 10 then close else 0 end)/sum(case when month = 10 then 1 else 0 end), 0) as oct_stock,
                   if(sum(case when month = 11 then 1 else 0 end) <> 0, sum(case when month = 11 then close else 0 end)/sum(case when month = 11 then 1 else 0 end), 0) as nov_stock,
                   if(sum(case when month = 12 then 1 else 0 end) <> 0, sum(case when month = 12 then close else 0 end)/sum(case when month = 12 then 1 else 0 end), 0) as dec_stock

            FROM test_3_1
            GROUP BY org, year
            Having jan_stock != 0
            AND feb_stock !=0
            AND mar_stock !=0
            AND apr_stock !=0
            AND may_stock !=0
            AND jun_stock !=0
            AND jul_stock !=0
            AND aug_stock !=0
            AND sep_stock !=0
            AND oct_stock !=0
            AND nov_stock !=0
            AND dec_stock !=0
"""

filter_3_stocks_sdf = spark.sql(query)
filter_3_stocks_sdf.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|              org|year|jan_stock|feb_stock|mar_stock|apr_stock|may_stock|jun_stock|jul_stock|aug_stock|sep_stock|oct_stock|nov_stock|dec_stock|
+-----------------+----+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
| Barclays Capital|2008|   29.806|   29.443|   27.993|   29.727|   26.613|   20.233|   19.248|   20.818|   20.253|   13.134|    8.101|    7.177|
|    Cisco Systems|2006|     15.4|   15.993|   17.564|   17.384|   17.121|   16.376|   15.173|   16.488|   18.727|   19.953|   21.578|   22.486|
|  GlaxoSmithKline|2006|   35.695|   35.537|   37.019|   36.732|   39.283|   37.995|   38.561|   38.104|   38.117|   37.948|    36.38|   36.522|
|           Oracle|2004|   13.107|   12.431|   11.142|    11.22|   10.511|   10.483|    9.843|     9.51|    9.947|   11.274|   11.

In [ ]:
#################     DO NOT EDIT      ##################
%%spark
filter_3_stocks_sdf.createOrReplaceTempView("test_3_2")
test_3_2_sdf = spark.sql("SELECT * FROM test_3_2 ORDER BY org, year ASC LIMIT 10")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#Convert to Pandas
%spark -o test_3_2_sdf

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
## AUTOGRADER 
grader.grade(test_case_id = 'test_ravel_sdf', answer = test_3_2_sdf)

Correct! You earned 8/8 points. You are a star!

Your submission has been successfully recorded in the gradebook.


### 3.3: Direction of Change [6 Pts]

The final element in our training set is the binary output for each case, i.e. the `y` label. 

Create an sdf `stocks_train_sdf` from `filter_3_stocks_sdf` with an additional column `direction`. This should be the direction of percentage change in the closing stock price, i.e. `1` for positive or `-1` for negative, from the first month of a given year to the last month of the given year. Make this an **integer**.  The year begins in January and ends in December, inclusive. The format of the sdf is shown below:

```
+----+-----+----------+---------+----------+-------------+
|org |year |jan_stock |   ...   |dec_stock |direction    |
+----+-----+----------+---------+----------+-------------+
|IBM |2008 |...       |   ...   |...       |1            |
|IBM |2009 |...       |   ...   |...       |-1           |
|... |...  |...       |   ...   |...       |...          |
+----+-----+----------+---------+----------+-------------+
```

In [ ]:
%%spark

#ToDo

query = """
            SELECT *, IF(dec_stock>jan_stock, 1, -1) as direction
            FROM test_3_2
"""


stocks_train_sdf = spark.sql(query)
stocks_train_sdf.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|              org|year|jan_stock|feb_stock|mar_stock|apr_stock|may_stock|jun_stock|jul_stock|aug_stock|sep_stock|oct_stock|nov_stock|dec_stock|direction|
+-----------------+----+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
| Barclays Capital|2008|   29.806|   29.443|   27.993|   29.727|   26.613|   20.233|   19.248|   20.818|   20.253|   13.134|    8.101|    7.177|       -1|
|    Cisco Systems|2006|     15.4|   15.993|   17.564|   17.384|   17.121|   16.376|   15.173|   16.488|   18.727|   19.953|   21.578|   22.486|        1|
|              IBM|2012|  157.075|  164.599|  172.858|  173.182|  170.101|  165.354|  162.753|  169.054|  174.401|  173.441|  163.985|  162.915|        1|
|           Pfizer|2010|   14.877|    14.07|   13.542|   13.218|   12.

In [ ]:
#################     DO NOT EDIT      ##################
%%spark
stocks_train_sdf.createOrReplaceTempView("test_3_3")
test_3_3_sdf = spark.sql("SELECT * FROM test_3_3 ORDER BY org, year ASC LIMIT 10")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#Convert to Pandas
%spark -o test_3_3_sdf

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
## AUTOGRADER 
grader.grade(test_case_id = 'test_train_sdf', answer = test_3_3_sdf)

Correct! You earned 6/6 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## Step 4: Combining LinkedIn and Stocks Data [10 pts]

### 4.1: The CRAZY Combination

Now that we have individually created the two halves of our training data we will merge them together to create the combined final training sdf.

Create an sdf called `training_sdf` in the format of the one shown at the beginning of Step 3. Note that in our definition for the `stock_result` column, the `stock_result` value for a particular year corresponds to the direction of the stock percentage change in the **following** year. For example, the stock_result in the `2008` row for `IBM` will contain the direction of IBM's stock in the year 2009. For the final training dataframe, we only need the entries for the companies where both hiring and stock data are available for the particular year.
The format of the sdf is shown below:
```
+----+-----+----------+---------+----------+----------+---------+----------+-------------+
|org |year |jan_hired |   ...   |dec_hired |jan_stock |   ...   |dec_stock |stock_result |
+----+-----+----------+---------+----------+----------+---------+----------+-------------+
|IBM |2008 |...       |   ...   |...       |...       |   ...   |...       |-1           |
|IBM |2009 |...       |   ...   |...       |...       |   ...   |...       |1            |
|... |...  |...       |   ...   |...       |...       |   ...   |...       |...          |
+----+-----+----------+---------+----------+----------+---------+----------+-------------+
```

In [ ]:
%%spark

#ToDo

query = """WITH stock as (SELECT * FROM
           (SELECT test_3_3.org, test_3_3.year,test_3_3.jan_stock,test_3_3.feb_stock,test_3_3.mar_stock,test_3_3.apr_stock,
                  test_3_3.may_stock,test_3_3.jun_stock,test_3_3.jul_stock,test_3_3.aug_stock,test_3_3.sep_stock,
                  test_3_3.oct_stock,test_3_3.nov_stock,test_3_3.dec_stock,
                  lead(test_3_3.direction) over(partition by test_3_3.org order by test_3_3.year) as stock_result
           FROM test_3_3)
           WHERE stock_result is not null)

           SELECT test_2_3.org, test_2_3.year, test_2_3.jan_hired, test_2_3.feb_hired, test_2_3.mar_hired, test_2_3.apr_hired, 
                  test_2_3.may_hired, test_2_3.jun_hired, test_2_3.jul_hired, test_2_3.aug_hired, 
                  test_2_3.sep_hired, test_2_3.oct_hired, test_2_3.nov_hired, test_2_3.dec_hired,
                  stock.jan_stock,stock.feb_stock,stock.mar_stock,stock.apr_stock,
                  stock.may_stock,stock.jun_stock,stock.jul_stock,stock.aug_stock,stock.sep_stock,
                  stock.oct_stock,stock.nov_stock,stock.dec_stock,stock.stock_result
           FROM stock, test_2_3
           WHERE test_2_3.org = stock.org
           AND test_2_3.year = stock.year
"""

training_sdf = spark.sql(query)
training_sdf.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+------------+
|                 org|year|jan_hired|feb_hired|mar_hired|apr_hired|may_hired|jun_hired|jul_hired|aug_hired|sep_hired|oct_hired|nov_hired|dec_hired|jan_stock|feb_stock|mar_stock|apr_stock|may_stock|jun_stock|jul_stock|aug_stock|sep_stock|oct_stock|nov_stock|dec_stock|stock_result|
+--------------------+----+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+------------+
|               Nokia|2011|       10|        3|        3|        4|        0|        1|        2|        3|        5|        6|        4|        1|    8.152|

In [ ]:
#################     DO NOT EDIT      ##################
%%spark
training_sdf.createOrReplaceTempView("test_4")
test_4_sdf = spark.sql("SELECT * FROM test_4 ORDER BY org, year ASC LIMIT 10")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#Convert to Pandas
%spark -o test_4_sdf

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
## AUTOGRADER 
grader.grade(test_case_id = 'test_merged_training_sdf', answer = test_4_sdf)

Correct! You earned 10/10 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## Step 5: Twitter followers [27 Pts]

In this section, we will be working with a dummy dataset representing connections on twitter, where each row consists of a person A, and a person B, such that person A follows person B. 

Let's introduce a fun little concept called the Bacon Number! The Bacon number of an actor or actress is the number of degrees of separation they have from actor Kevin Bacon, as defined by the game known as Six Degrees of Kevin Bacon. For example, Kevin Bacon's Bacon number is 0. If an actor works in a movie with Kevin Bacon, the actor's Bacon number is 1. If an actor works with an actor who worked with Kevin Bacon in a movie, the first actor's Bacon number is 2, and so forth.

How do we implement the "Super Bacon" for our dataset though? We define this number as follows: if person A follows person B, and person B follows person C, then the super bacon of C with respect to A will be 2.

Now to calculate this number, we'll use the concepts of graphs and BFS!

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 18.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=f35ec968244fe9c885327d3cf35eda947bc938616a0ac5ec2f7c8659b5e8f689
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

appName = "PySpark"

# Create Spark session
spark = SparkSession.builder.appName(appName).getOrCreate()

### 5.1: “Traversing” a Graph

Let's review how BFS works!

In [ ]:
import pandas as pd
from IPython.display import Image as I

bfsgif =\
'https://upload.wikimedia.org/wikipedia/commons/5/5d/Breadth-First-S'+\
'earch-Algorithm.gif'
dfsgif=\
'https://upload.wikimedia.org/wikipedia/commons/7/7f/Depth-First-Search.gif'


#### 5.1.1 Intro to Distributed Breadth-First Search


To start off, we will be implementing a graph traversal algorithm known as Breadth First Search. It works in a way that's equivalent to how a stain spreads on a white t-shirt. Take a look at the graph below:

<p align = "center">
<img src = "https://imgur.com/WU3AUwg.png" width= "600" align ="center"/>

* Consider starting BFS from point A (green). This is considered the starting frontier/singular origin node.
* The first round of BFS would involve finding all the nodes directly reachable from A, namely B-F (blue circles). These blue nodes make up the next frontier at depth 1 away from our starting node A.
* The second round would then be identifying the red nodes which are the neighbors of the blue nodes. Now, the red nodes all belong to a frontier 2 depth away from A. Note that node A is also a neighbor of a blue node. However, since it has already been visited, it does not get added to this frontier.

This process continues until all the nodes in the graph have been visited.
If you would like to learn more about BFS, we highly suggest looking [here](https://www.tutorialspoint.com/data_structures_algorithms/breadth_first_traversal.htm).


We will now be implementing **spark_bfs(G, N, d)**, our spark flavor of BFS that takes a graph **G**, a set of origin nodes **N**, and a max depth **d**.

In order to write a successful BFS function, you are going to need to figure out 
1. how to keep track of nodes that we have visited
2. how to properly find all the nodes at the next depth
3. how to avoid cycles and ensure that we do not constantly loop through the same edges (take a look at J-K in the graph)

#### [OPTIONAL/ADDITIONAL] BFS vs. DFS Animation 
Run the code cells below to understand the difference between depth and breadth first search! (Source: Wikimedia Commons)

In [ ]:
# NOT GRADED
print('BFS:')
I(url=bfsgif)

BFS:


In [ ]:
#NOT GRADED
print('DFS:')
I(url=dfsgif)

DFS:


#### 5.1.2 Implement One Traversal [6 Pts]

To break down this process, let's think about how we would implement a single traversal of the graph. That is given the green node in the graph above, how are we going to get the blue nodes?


Consider the simple graph below **which is different from the graph in the image above**:

In [ ]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

simple = [('A', 'B'),
         ('A', 'C'),
         ('A', 'D'),
         ('C', 'F'),
         ('F', 'A'),
         ('B', 'G'),
         ('G', 'H'),
         ('D', 'E')]
         
simple_dict = {'from_node': ['A', 'A', 'A', 'C', 'F', 'B', 'G', 'D'],
       'to_node': ['B', 'C', 'D', 'F', 'A', 'G', 'H', 'E']}

simple_graph_df = pd.DataFrame.from_dict(simple_dict)
simple_graph_sdf = spark.createDataFrame(simple_graph_df)
simple_graph_sdf.show()

+---------+-------+
|from_node|to_node|
+---------+-------+
|        A|      B|
|        A|      C|
|        A|      D|
|        C|      F|
|        F|      A|
|        B|      G|
|        G|      H|
|        D|      E|
+---------+-------+



As you can see, each row of this dataframe represents an edge between two nodes. Here, we are looking at a directed graph, which means that A-->B does not represent the same edge as B-->A.
Let's define our starting node as follows:

In [ ]:
smallOrig = [{'node': 'A'}]

Then, bfs with graph G, starting from smallOrig to depth 1, or spark_bfs(G, smallOrig, 1) would output as follows:

In [ ]:
simple_1_round_dict = {'node': ['B', 'D', 'C', 'A'],
       'distance': [1, 1, 1, 0]}
simple_1_round_bfs_df = pd.DataFrame.from_dict(simple_1_round_dict)
simple_1_round_bfs_sdf = spark.createDataFrame(simple_1_round_bfs_df)
simple_1_round_bfs_sdf.show()

+----+--------+
|node|distance|
+----+--------+
|   B|       1|
|   D|       1|
|   C|       1|
|   A|       0|
+----+--------+



As you can see, this dataframe logs each node with its corresponding distance away from A. Moreover, we also know that these nodes are **visited**. 

Hopefully, you can see how we can use our original graph and this new information to find the nodes at depth two. 

This is exactly what we will try to accomplish with **spark_bfs_1_round(visited_nodes)** which will ultimately be the inner function of **spark_bfs** that we use to perform exactly one traversal of a graph.

**TODO**: Write **spark_bfs_1_round(visited_nodes)** that takes the currently dataframe of visited_nodes, performs one round of BFS, and returns an updated visited nodes dataframe. You should assume that a temporary sdf `G` already exists.

In [ ]:
def spark_bfs_1_round(visited_nodes):
  """
  :param visited_nodes: dataframe with columns node and distance
  :return: dataframe of updated visited nodes, with columns node and distance
  """
  # TODO: Complete this function to implement 1 round of BFS
  visited_nodes = visited_nodes.toPandas()
  max_dis = visited_nodes['distance'].max()
  new_node = []
  for item in visited_nodes['node']:
    if int(visited_nodes[visited_nodes['node']==item]['distance']) == max_dis:
      new_node.extend(list(simple_graph_df[simple_graph_df['from_node']==item]['to_node']))
  
  new_dict = {'node': new_node,
       'distance': [max_dis+1]*len(new_node)}
  new_dict_df = pd.DataFrame.from_dict(new_dict)

  return spark.createDataFrame(pd.concat([new_dict_df,visited_nodes]))

Now, run the inner function on simple_1_round_bfs_sdf (i.e. result of 1 round of BFS on the simple graph) and store the results in simple_bfs_result. This is ultimately what the output of BFS to depth 2 should look like. Create your temporary sdf `G` and then call the `spark_bfs_1_round` function you created above.

In [ ]:
# TODO: Run spark_bfs_1_round on simple_1_round_bfs_sdf
simple_bfs_result = spark_bfs_1_round(simple_1_round_bfs_sdf)

Convert this result to Pandas, sort your dataframe by `node` ascending, and submit it to the autograder.

In [ ]:
# TODO: Convert simple_bfs_result to Pandas
simple_bfs_test = simple_bfs_result.toPandas()

In [ ]:
###DO NOT EDIT
simple_bfs_test = simple_bfs_test.sort_values(by=["node"],ascending=True)

In [ ]:
# 6 points
grader.grade(test_case_id = 'checksimpleBFS', answer = simple_bfs_test)

Correct! You earned 6/6 points. You are a star!

Your submission has been successfully recorded in the gradebook.


#### 5.1.3 Full BFS Implementation

Now, we will fully implement **spark_bfs**. This function should iteratively call your implemented version of **spark_bfs_1_round** and ultimately return the output of this function at **max_depth**.

You are also responsible for initializing the starting dataframe, that is converting the list of origin nodes into a spark dataframe with the nodes logged at distance 0.

Consider the following: 

```
schema = StructType([
            StructField("node", StringType(), True)
        ])

    my_sdf = spark.createDataFrame(origins, schema)
```

The schema ultimately specifies the structure of the Spark DataFrame with a string `node` column. It then calls **spark.createDataFrame** to map this schema to the **origins** nodes. Also, you are responsible for ensuring that a view of your graph is available within this function. (Note: you will also need to add in a distance column)

**TODO:** implement **spark_bfs(G,origins,max_depth)**. In the coming sections, you will run this function on **twitter_sdf** that you will initialize in 5.2. Note: you may want to run tests on the **simple_graph** example as the `twitter_sdf` will take quite some time to run.

Also remember that in section 5.1.2, we had assumed that the temporary view already exists. So you will have to take care of creating that within this function.

These imports might be useful: 
`from pyspark.sql.types import StructType, StructField, StringType, IntegerType`

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
# TODO: iterative search over undirected graph
# Worth 5 points directly, but will be needed later

def spark_bfs(G, origins, max_depth):
  """ runs distributed BFS to a specified max depth

  :param G: graph dataframe from 4.3
  :param origins: list of origin nodes stored as {"node": nodeValue}
  :param max_depth: integer value of max depth to run BFS to
  :return: dataframe with columns node, distance of all visited nodes
  """
  depth = 0
  origins[0]['distance'] = 0
  schema = StructType([
            StructField("node", StringType(), True),
            StructField("distance", IntegerType(), True)
        ])
  visited_nodes = spark.createDataFrame(origins, schema)
  while depth!= max_depth:
    visited_nodes = spark_bfs_1_round(visited_nodes)
    depth += 1
  return visited_nodes

Test that this function works on the simple example.

In [ ]:
simple_bfs_iterative_result = spark_bfs(simple_graph_sdf, smallOrig, 3)
simple_bfs_iterative_result.show()

+----+--------+
|node|distance|
+----+--------+
|   H|       3|
|   A|       3|
|   G|       2|
|   F|       2|
|   E|       2|
|   B|       1|
|   C|       1|
|   D|       1|
|   A|       0|
+----+--------+



### 5.2: Popular People [6 Pts]

Download the `twitter_followers.csv` dataset from [here](https://drive.google.com/file/d/1MNGXvH2kxanW1iOOp1egz6cMg_cugxE9/view?usp=share_link), and upload it to the content folder of this Colab notebook.

It's time to take a look at our dataset. Let's load the data to **twitter_sdf** and see how it looks.

In [ ]:
twitter_sdf = spark.read.csv("/content/twitter_followers.csv", header=True)

In [ ]:
twitter_sdf.show()

+---+-----------+---------+
|_c0|      User1|    User2|
+---+-----------+---------+
|  0|      James|   Ashley|
|  1|      Karen|   Thomas|
|  2|     Sandra|    Karen|
|  3|      Karen|   Joseph|
|  4|   Patricia| Margaret|
|  5|      Karen|  Charles|
|  6|    Charles| Patricia|
|  7|Christopher|   Ashley|
|  8|  Elizabeth|    Karen|
|  9|    Barbara|  Anthony|
| 10|       Mark|  Michael|
| 11|    Anthony|  Jessica|
| 12|  Elizabeth|    Karen|
| 13|    Michael|  Richard|
| 14|    Anthony|  Charles|
| 15|      Karen|   Sandra|
| 16|      Karen| Margaret|
| 17|     Robert|   Daniel|
| 18|      Karen|    David|
| 19|Christopher|Elizabeth|
+---+-----------+---------+
only showing top 20 rows



As we can see, the dataframe consists of 2 columns named User1 and User2. These columns indicate an edge in the social network graph formed when people follow each other on twitter. Each row in the dataframe indicates that user1 follows user2. For example, the 0 indexed row has James as User1 and Ashley as User2, which indicates that James follows Ashley.

Let's take a look at which people were the most popular in terms of the number of followers they had. ***TODO***: Count the number of followers each person (Consider only those people who have atleast one follower. i.e. Those whose name exists in the User2 column). Return a dataframe with the schema (Name,Followers). Order your dataframe by Followers in descending order.

In [ ]:
# TODO
twitter_df = twitter_sdf.toPandas()
new_dict = {}
for item in twitter_df['User2']:
  new_dict[item] = len(twitter_df[twitter_df['User2']==item])

In [ ]:
new_dict

Convert the dataframe to pandas and pass it to the grader.

In [ ]:
# TODO
count_df = pd.DataFrame({'Name':new_dict.keys(),'Followers':new_dict.values()})
count_df.sort_values(by='Followers',ascending=False,inplace=True)

In [ ]:
# 4 points
grader.grade(test_case_id = 'checkFollowerCount', answer = count_df)

You earned 3/4 points.

But, don't worry you can re-submit and we will keep only your latest score.


Now that we have the Twitter dataset, let's convert it to a graph sdf just like the one we had in section 5.1 (P.S. it's not as hard as it sounds).

**TODO:** Create **twitter_graph_sdf** that has the columns *from_node* and *to_node*. from_node has all the entries from the *User1* column and to_node has all the entries from the *User2* column.

In [ ]:
# TODO


In [ ]:
grader.grade(test_case_id = 'checkTwitterGraph', answer = twitter_graph_sdf.toPandas())

### 5.3: The Super Bacon Search [15 Pts]

In the previous section, we found out that Patricia and Karen have been the most popular people for tour dataset. So let's find out the Super Bacons with respect to Karen.

In [ ]:
orig = [{'node': 'Karen'}]

In [ ]:
bfs_5 = spark_bfs(twitter_graph_sdf, orig, 5)
bfs_5.show()

In [ ]:
answer_df = bfs_5.toPandas()

In [ ]:
# 15 points
grader.grade(test_case_id = 'checkBFS', answer = answer_df)

# HW Submission

Before you submit on Gradescope (you must submit your notebook to receive credit):


1.   Restart and Run-All to make sure there's nothing wrong with your notebook
2.   **Double check that you have the correct PennID (all numbers) in the autograder**. 
3. Make sure you've run all the PennGrader cells
4. Go to the "File" tab at the top left, and click "Download .ipynb" and then "Download .py".  **Rename** the files to "homework3.ipynb" and "homework3.py" respectively and upload them to Gradescope 

**Let the course staff know ASAP if you have any issues submitting, but otherwise best of luck!**